In [2]:
# -- run me first --
from pprint import pprint  # for pretty printing
# display all outputs, not only last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
print("-done-")

-done-


<center>🐍</center>

***
# 6. Классы и ООП. Часть 2
<div style="text-align: right; font-weight: bold">Aleksandr Koriagin</div>
<div style="text-align: right; font-weight: bold"><span style="color: #76CDD8;">&lt;</span>epam<span style="color: #76CDD8;">&gt;</span></div>
<div style="text-align: right; font-weight: bold">May 2020</div>
<div style="text-align: right; font-style: italic">Nizhny Novgorod</div>

***
## Оглавление<a id="0"></a>

1. [Магические методы](#1)
    1. `getattr` и `__getattr__`
    1. `setattr` и `__setattr__`
    1. Операторы сравнения
    1. `__call__`
    1. `__repr__` и `__str__`
    1. `__hash__`
2. [Дескрипторы](#2)
    1. Зачем нам нужны дексрипторы?
    1. Как использовать дескрипторы
    1. Дескрипторы Python 3.6+
    1. Подробнее про get, set, delete
    1. Резюме
    1. Практическая работа
3. [Классы-примеси (mixins)](#3)
4. [Модуль ABC (abstract base classes)](#4)
    1. Практическая работа
5. [Метаклассы](#5)
    1. Классы как объекты
    1. Динамическое создание классов
    1. Что такое метакласс (наконец)
    1. Атрибут `__metaclass__`
    1. Пользовательские метаклассы
    1. Зачем использовать метаклассы вместо функций?
    1. Зачем вообще использовать метаклассы?
    1. Напоследок
6. [Домашнее задание](#6)

In [ ]:
%%bash
# generate table of contents
cat 6_classes_and_oop_part-2.ipynb | grep "##" | grep -v "cat" | sed  "s/#/    /g" | tr -d '"'

***
## 1. Магические методы<a id="1"></a>

"Магическими" называются внутренние методы классов, например, метод `__init__`.

С помощью "магических" методов можно:
* управлять доступом к атрибутам экземпляра, 
* перегрузить операторы, например, операторы сравнения или арифметические операторы, 
* определить строковое представление экземпляра или изменить способ его хеширования.

Мы рассмотрим только часть наиболее используемых методов.
<br>Подробное описание всех “магических” методов можно найти в документации языка. 

Дополнительно: [Special method names](https://docs.python.org/3/reference/datamodel.html#special-method-names)

### `getattr` и `__getattr__`

Метод `__getattr__` вызывается при попытке прочитать значение несуществующего атрибута

In [2]:
class Nope:
    bar = 123
    def __getattr__(self, name):
        return f"No such name: '{name}'"

nope = Nope()
nope.bar
nope.foo

123

"No such name: 'foo'"

Функция `getattr` позволяет безопасно получить значение атрибута экземпляра класса по его имени

In [3]:
class A:
    var_a = 1
a = A()

getattr(a, "var_a")
getattr(a, "b", "default value")
#getattr(a, "b")

aa = {"a": 1}
aa.get("a")
print(aa.get("bbbbb"))

1

'default value'

1

None


### `setattr` и `__setattr__`

Метод `__setattr__` позволяет управлять изменением значения.
<br>В отличие от `__getattr__`, он вызываются для всех атрибутов, а не только для несуществующих.

Во избежание рекурсии реализация метода не должна пытаться присвоить атрибут объекту обычным путём: `self.name = value.`
<br>Вместо этого следует добавить атрибут в словарь атрибутов объекта, например: `self.__dict__[name] = value`.
<br>При этом для классов нового стиля предпочтительно обратиться к методу базового класса: `object.__setattr__(self, name, value)`.

In [ ]:
class AccessControl:
    permitted = ("age")

    def __setattr__(self, attr, value):
        if attr in self.permitted:
            object.__setattr__(self, attr, value)  # self.__dict__[attr] = value
        else:
            raise AttributeError(f"'{attr}' not allowed")

x = AccessControl()
x.age = 40                # Вызовет метод __setattr__
x.age
x.name = 'pythonlearn'  # AttributeError

Функция `setattr` добавляет атрибут:

In [4]:
class AttrHolder:
    pass
holder = AttrHolder()

for attr in ("aaa", "bbb", "ccc", "ddd"):
    setattr(holder, attr, f"value_{attr}")

vars(holder)
holder.bbb

holder.asdsadad = 123

{'aaa': 'value_aaa',
 'bbb': 'value_bbb',
 'ccc': 'value_ccc',
 'ddd': 'value_ddd'}

'value_bbb'

In [5]:
delattr(holder, "aaa")
vars(holder)

{'bbb': 'value_bbb', 'ccc': 'value_ccc', 'ddd': 'value_ddd', 'asdsadad': 123}

### Операторы сравнения

Чтобы экземпляры класса поддерживали все операторы сравнения, нужно реализовать внушительное количество "магических" методов:
```python
instance.__eq__(other)  # instance == other
Instance.__ne__(other)  # instance != other
instance.__lt__(other)  # instance < other
instance.__le__(other)  # instance <= other
instance.__gt__(other)  # instance > other
instance.__ge__(other)  # instance >= other
```

В уже знакомом нам модуле `functools` есть декоратор, облегчающий реализацию операторов сравнения: [functools.total_ordering](https://docs.python.org/3.8/library/functools.html#functools.total_ordering)

In [ ]:
import functools
@functools.total_ordering
class Counter:
    def __init__(self, value):
        self.value = value

    def __eq__(self, other) -> bool:
        return self.value == other.value

    def __lt__(self, other) -> bool:
        return self.value < other.value

Counter(1) > Counter(2)
vars(Counter)

### `__call__`

Метод `__call__` позволяет "вызывать" экземпляры классов, имитируя интерфейс функций

In [6]:
class Useful:
    def __call__(self, x):
        return x

Useful()(42)

42

In [10]:
# Декоратор с аргументами на основе класса
import functools
import sys

class Trace:
    def __init__(self, handle):
        self.handle = handle

    def __call__(self, func):
        @functools.wraps(func)
        def inner(*args, **kwargs):
            print(f"Name={func.__name__}, Args={args}, Kwargs={kwargs}", file=self.handle)
            return func(*args, **kwargs)
        return inner

@Trace(handle=sys.stderr)
def useful(x, y):
    return x + y

useful(11, 22)

Name=useful, Args=(11, 22), Kwargs={}


33

### `__repr__` и `__str__`

В Python есть две различных по смыслу функции для преобразования объекта в строку: `repr` и `str`.
<br>Для каждой из них существует одноимённый "магический" метод:

In [12]:
class Counter:
    def __init__(self, initial=0):
        self.value = initial

    def __repr__(self):
        return "Counter({})".format(self.value)

    def __str__(self):
       return "Counted to {}".format(self.value)

counter = Counter(44)
counter
print(counter)

Counter(44)

Counted to 44


### `__hash__`

Метод `__hash__` используется для вычисления значения хеш-функции.
<br>Реализация по умолчанию гарантирует, что одинаковое значение хеш функции будет только у физически одинаковых объектов, то есть:
```
x is y <=> hash(x) == hash(y)
```
Несколько очевидных рекомендаций:
* Метод `__hash__` имеет смысл реализовывать только вместе с методом `__eq__`.
<br>При этом реализация `__hash__` должна удовлетворять: `x == y <=> hash(x) == hash(y)`
* Для изменяемых объектов можно ограничиться только методом `__eq__`.

***
## 2. Дескрипторы<a id="2"></a>

Дескриптор это атрибут объекта со "связанным поведением", то есть такой атрибут, при доступе к которому его поведение переопределяется методом протокола дескриптора. Эти методы `__get__`, `__set__` и `__delete__`. Если хотя бы один из этих методов определен в объекте, то можно сказать что этот метод дескриптор.

В Python существует три варианта доступа к атрибуту.
<br>Допустим у нас есть атрибут `a` объекта `obj`:
* Получим значение атрибута: `some_variable = obj.a`
* Изменим его значение: `obj.a = 'new value'`
* Удалим атрибут: `del obj.a`

Python позволяет перехватить выше упомянутые попытки доступа к атрибуту и переопределить связанное с этим доступом поведение. Это реализуется через механизм протокола дескрипторов.

### Зачем нам нужны дексрипторы?

Давайте рассмотрим пример:

In [ ]:
class Order:
    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity
    def total(self):
        return self.price * self.quantity

apple_order = Order('apple', 1, 10)
apple_order.total()

Если этот код начать использовать, мы столкнемся с проблемой.
<br>Наши данные ни как не проверяются.
<br>То есть цена `price` и количество `quantity` может принимать любое значение:

In [ ]:
apple_order.quantity = -10
apple_order.total()

Вместо того чтобы использовать методы `getter` и `setter` и создавать новое API, давайте используем стандартный декоратор `@property` для проверки значения атрибута `quantity`:

In [ ]:
class Order:
    def __init__(self, name, price, quantity):
        self.price = price
        self._quantity = quantity  # (1)

    @property
    def quantity(self):
        return self._quantity

    @quantity.setter
    def quantity(self, value):
        if value < 0:
            raise ValueError('Cannot be negative.')
        self._quantity = value     # (2)

    def total(self):
        return self.price * self.quantity

apple_order = Order('apple', 1, 10)
apple_order.quantity = 15
apple_order.total()
# apple_order.quantity = -10  # ValueError: Cannot be negative

Мы преобразовали `quantity` из простого атрибута в неотрицательное свойство.
<br>Обратите внимание на строку **(1)** где мы переименовали атрибут в `_quantity` что бы избежать получение в строке **(2)** ошибки `RecursionError`.

Мы забыли об атрибуте `price`, который также не может быть отрицательным.
<br>Возможно, первым делом вы пытаетесь просто скопировать то что мы делали для атрибута `_quantity`.
<br>А что если у нас будет двадцать таких атрибутов.
<br>Помните принцип DRY *(Don't repeat yourself)*: когда вы обнаруживаете, что делаете одно и то же дважды, это хороший признак для удаления повторно используемого кода.
<br>Давайте посмотрим, чем нам в этом случае могут помочь дескрипторы.

### Как использовать дескрипторы

При использовании дескрипторов наше новое определение класса станет таким:

In [ ]:
class NonNegative:
    def __init__(self, name):
        self.name = name                      # (4)

    def __get__(self, instance, owner):
        return instance.__dict__[self.name]   # (5)

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Cannot be negative.")
        instance.__dict__[self.name] = value  # (6)

class Order:
    price = NonNegative(name="price")         # (3)
    quantity = NonNegative(name="quantity")

    def __init__(self, name, price, quantity):
        self.name = name
        self.price = price
        self.quantity = quantity
    
    def total(self):
        return self.price * self.quantity

apple_order = Order("apple", 1, 10)
apple_order.total()
# apple_order.price = -10     # ValueError: Cannot be negative
# apple_order.quantity = -10  # ValueError: Cannot be negative

Строка **(4)**: атрибут `name` необходим при создание объекта `NonNegative` в строке **(3)**.
<br>Таким образом, мы явно передаем имя атрибута `price` что бы использовать его как ключ при доступе к экземпляру `__dict__`.

### Дескрипторы Python 3.6+

В Python 3.6 появился [новый протокол дескрипторов](https://docs.python.org/3.6/howto/descriptor.html#definition-and-introduction):

`object.__set_name__(self, owner, name)` - Вызывается во время создания класса. В этом случае дескриптор назначается на имя атрибута.

С этим протоколом, мы можем удалить `__init__` у `NonNegative` и привязать имя атрибута к дескриптору:

In [ ]:
class NonNegative:
    def __get__(self, instance, owner):
        # print(instance)  # <__main__.Order object at 0x7fa7f065f390>
        # print(owner)     # <class '__main__.Order'>
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        # print(instance)  # <__main__.Order object at 0x7fa7f065f8d0>
        if value < 0:
            raise ValueError("Cannot be negative.")
        instance.__dict__[self.name] = value

    def __set_name__(self, owner, name):
        # print(owner)  # <class '__main__.Order'>
        # print(name)
        self.name = name

class Order:
    price = NonNegative()
    quantity = NonNegative()

    def __init__(self, name: str, price: int, quantity: int):
        self.name = name
        self.price = price
        self.quantity = quantity

    def total(self):
        return self.price * self.quantity

apple_order = Order("apple", 1, 10)
#apple_order.total()

apple_order.price = 1

# apple_order.price = -10     # ValueError: Cannot be negative
# apple_order.quantity = -10  # ValueError: Cannot be negative

### Подробнее про get, set, delete

**get**:
<br>Метод `__get__` вызывается при доступе к атрибуту.
<br>Метод принимает два аргумента:
* `instance` — экземпляр класса или `None`, если дескриптор был вызван в результате обращения к атрибуту у класса
* `owner` — класс, "владеющий" дескриптором.

**set**:
<br>Метод `__set__` вызывается для изменения значения атрибута.
<br>Метод принимает два аргумента:
* `instance` - экземпляр класса, “владеющего” дескриптором,
* `value` - новое значение атрибута.

**delete**:
<br>Метод `__delete__` вызывается при удалении атрибута.
<br>Метод принимает один аргумент — экземпляр класса, “владеющего” дескриптором.

### Резюме

* Как и свойства, дескрипторы позволяют контролировать чтение, изменение и удаление атрибута, но, в отличие от свойств, дескрипторы можно переиспользовать.
* Дескриптор — это экземпляр класса, реализующего любую комбинацию методов `__get__`, `__set__` и `__delete__`.

Все дескрипторы можно поделить на две группы:
* дескрипторы данных `data descriptors`, определяющие как минимум метод `__set__`
* остальные `non-data descriptors`

Полезные дескрипторы определяют ещё и метод `__get__`.

**Дополнительно:** [Руководство к дескрипторам](https://habr.com/ru/post/122082/), [Атрибуты и протокол дескриптора в Python](https://habr.com/ru/post/479824/)

### <font color='blue'><u>Практическая работа</u></font>

Создать класс `Book` с аттрибутами `price`, `author`, `name`.
<br>Автор и название книги не должны меняться. (Выкидываем `ValueError`)
<br>Цена может меняться, но должна находится в пределах: `0 <= price <= 100`

``` python
>>> b = Book("William Faulkner", "The Sound and the Fury", 12)

>>> print(f"Author='{b.author}', Name='{b.name}', Price='{b.price}'")
Author='William Faulkner', Name='The Sound and the Fury', Price='12'

>>> b.price = 55
>>> b.price
55
>>> b.price = -12  # => ValueError: Price must be between 0 and 100.
>>> b.price = 101  # => ValueError: Price must be between 0 and 100.

>>> b.author = "new author"  # => ValueError: Author can not be changed.
>>> b.name = "new name"      # => ValueError: Name can not be changed.
```

***
## 3. Классы-примеси (mixins)<a id="3"></a>

Классы-примеси позволяют выборочно модифицировать поведение класса в предположении, что класс реализует некоторый интерфейс.

In [ ]:
import logging

class EssentialFunctioner:

    def __init__(self):
        self.logger = logging.getLogger(f"{self.__module__}.{self.__class__.__name__}")

    def do_the_thing(self):
        try:
            1 / 0
        except ZeroDivisionError:
            self.logger.exception("OH NO")

EssentialFunctioner().do_the_thing()

А что если понадобится создать 10 классов с одинаковым логгером?

In [ ]:
import logging

class LoggerMixin:
    @property
    def logger(self):
        name = f"{self.__module__}.{self.__class__.__name__}"
        return logging.getLogger(name)

class EssentialFunctioner(LoggerMixin):
    def do_the_thing(self):
        try:
            1 / 0
        except ZeroDivisionError:
            self.logger.exception("OH NO!")

class BusinessLogicer(LoggerMixin):
    def __init__(self):
        # super().__init__()
        self.logger.warning("\nBusinessLogicer init!")

EssentialFunctioner().do_the_thing()
BusinessLogicer()

***
## 4. Модуль ABC (abstract base classes)<a id="4"></a>

Абстрактным называется класс, который содержит один и более абстрактных методов.
<br>Абстрактным методом называется объявленный, но не реализованный метод.
<br>Абстрактные классы не могут быть инстанциированы, от них нужно унаследовать, реализовать все их абстрактные методы и только тогда можно создать экземпляр такого класса.
<br>Дополнительно: [Abstract Base Classes](https://docs.python.org/3/library/abc.html)

Если метод класса имеет декоратор `@abstractmethod`, значит его необходимо переопределить в классе-наследнике.

In [ ]:
from abc import ABC, abstractmethod

class A(ABC):
    @abstractmethod
    def some(self):
        return "some"

A().some()

Возьмем для примера, шахматы. 
<br>У всех шахматных фигур есть общий функционал, например - возможность фигуры ходить и быть отображенной на доске.
<br>Исходя из этого, мы можем создать абстрактный класс `Фигура`, определить в нем абстрактный метод (в нашем случае - `ход`, поскольку каждая фигура ходит по-своему) и реализовать общий функционал (`отрисовка` на доске).

In [ ]:
from abc import ABC, abstractmethod

class ChessPiece(ABC):
    def draw(self):
        # общий метод, который будут использовать все наследники этого класса
        return("Drew a chess piece")

    @abstractmethod
    def move(self):
        # абстрактный метод, который будет необходимо переопределять для каждого подкласса
        pass

class Queen(ChessPiece):
    def move(self):
        return("Moved Queen to e2e4")

q = Queen()
# Нам доступны все методы класса
q.draw()
q.move()    

Поведение абстрактных методов можно не только переопределять, но и дополнять:

In [ ]:
from abc import ABC, abstractmethod

class Basic(ABC):
    @abstractmethod
    def hello(self):
        print("> Hello from Basic class")

class Advanced(Basic):
    def hello(self):
        super().hello()
        print("> Enriched functionality")

a = Advanced()
a.hello()

Используя концепцию абстрактных классов, мы можем улучшить качество архитектуры приложения, уменьшить объем работы и при этом, обеспечить легкость дальнейшей поддержки кода.

Дополнительно: [Абстрактная фабрика](https://refactoring.guru/ru/design-patterns/abstract-factory)

### <font color='blue'><u>Практическая работа</u></font>

Создать абстрактный класс `Vehicle` с методами:
* `vehicle_type`, который выводит имя и тип транспортного средства (ТС),
* `is_motorcycle`, который выводит `True/False` в зависимости от числа колес *(2 колеса -> мотоцикл)*,
* `purchase_price` - выводит стоимость ТС в зависимости от кол-ва пройденных км: (`базовая цена - 0.1 * кол-во км`).
<br>Если получился меншьше "100" - вернуть "100".

Класс должен содержать поля: 
* год выпуска
* имя бренда
* все поля необходимые для обеспечения функциональности классов наследников.

Расставить где необходимо и если необходимо `abstractmethod`, `classmethod`, `staticmethod` или другие декораторы.

Создать классы наследники `Vehicle`: `Car`, `Motocycle`, `Truck`, `Bus`.

```python
vehicles = (
    Car(brand_name="Toyota", year_of_issue=2020, base_price=1_000_000, mileage=150_000),
    Motocycle(brand_name="Suzuki", year_of_issue=2015, base_price=800_000, mileage=35_000),
    Truck(brand_name="Scania", year_of_issue=2018, base_price=15_000_000, mileage=850_000),
    Bus(brand_name="MAN", year_of_issue=2000, base_price=10_000_000, mileage=950_000),
)

for vehicle in vehicles:
    print(
        f"Vehicle type={vehicle.vehicle_type()}\n"
        f"Is motorcycle={vehicle.is_motorcycle()}\n"
        f"Purchase price={vehicle.purchase_price()}\n"
    )
```
```
Vehicle type=Toyota Car
Is motorcycle=False
Purchase price=985000.0

Vehicle type=Suzuki Motocycle
Is motorcycle=True
Purchase price=796500.0

Vehicle type=Scania Truck
Is motorcycle=False
Purchase price=14915000.0

Vehicle type=MAN Bus
Is motorcycle=False
Purchase price=9905000.0
```

***
## 5. Метаклассы (optional)<a id="5"></a>

Статья: [Метаклассы в Python](https://habr.com/ru/post/145835/), Оригинал: [Classes as objects](https://stackoverflow.com/a/6581949/12551738)

>Метаклассы это глубокая магия, о которой 99% пользователей даже не нужно задумываться. Если вы думаете, нужно ли вам их использовать — вам не нужно (люди, которым они реально нужны, точно знают, зачем они им, и не нуждаются в объяснениях, почему).
><br>~ _Гуру Питона Тим Питерс_

### Классы как объекты

Перед тем, как изучать метаклассы, надо хорошо разобраться с классами, а классы в Питоне — вещь весьма специфическая (основаны на идеях из языка Smalltalk).

В большинстве языков класс это просто кусок кода, описывающий, как создать объект.
<br>В целом это верно и для Питона:

In [ ]:
class ObjectCreator:
    pass

my_object = ObjectCreator()
print(ObjectCreator)
print(my_object)

Но в Питоне класс это нечто большее — классы также являются объектами.

Как только используется ключевое слово `class`, Питон исполняет команду и создаёт объект.
<br>Инструкция
```python
class ObjectCreator:
    pass
```
создаст в памяти объект с именем `ObjectCreator`.

Этот объект (класс) сам может создавать объекты (экземпляры), поэтому он и является классом.

Тем не менее, это объект, а потому:
* его можно присвоить переменной,
* его можно скопировать,
* можно добавить к нему атрибут,
* его можно передать функции в качестве аргумента

### Динамическое создание классов

Так как классы являются объектами, их можно создавать на ходу, как и любой объект.

Например, можно создать класс в функции, используя ключевое слово `class`:

In [ ]:
def choose_class(name):
    if name == 'foo':
        class Foo:
            pass
        return Foo # возвращает класс, а не экземпляр
    else:
        class Bar:
            pass
        return Bar

MyClass = choose_class("foo")
print(MyClass)    # функция возвращает класс, а не экземпляр
print(MyClass())  # можно создать экземпляр этого класса

Однако это не очень-то динамично, поскольку по-прежнему нужно самому писать весь класс целиком.

Поскольку классы являются объектами, они должны генерироваться чем-нибудь.

Когда используется ключевое слово `class`, Python создаёт этот объект автоматически.
<br>Но как и большинство вещей в Питоне, есть способ сделать это вручную.

Помните функцию `type`?
<br>Старая-добрая функция, которая позволяет определить тип объекта:

In [3]:
class ObjectCreator:
    pass

type(1)
type("1")
type(ObjectCreator)
type(ObjectCreator())

int

str

type

__main__.ObjectCreator

На самом деле, у функции `type` есть совершенно иное применение: она также может создавать классы на ходу. 
<br>`type` принимает на вход описание класса и созвращает класс.

(Я знаю, это по-дурацки, что одна и та же функция может использоваться для двух совершенно разных вещей в зависимости от передаваемых аргументов. Так сделано для обратной совместимости)

`type` работает следующим образом:
```python
type(<имя класса>, 
     <кортеж родительских классов>, # для наследования, может быть пустым
     <словарь, содержащий атрибуты и их значения>
)
```
Например,
```python
class MyShinyClass:
    pass
```
может быть создан вручную следующим образом:

In [4]:
MyShinyClass = type("MyShinyClass", (), {"a": 123}) # возвращает объект-класс
print(MyShinyClass)
print(MyShinyClass())  # создаёт экземпляр класса

<class '__main__.MyShinyClass'>


Возможно, вы заметили, что мы используем `"MyShinyClass"` и как имя класса, и как имя для переменной, содержащей ссылку на класс. Они могут быть различны, но зачем усложнять?

`type` принимает словарь, определяющий атрибуты класса:
```python
class Foo:
    bar = True
```
можно переписать как:

In [5]:
Foo = type('Foo', (), {'bar':True})

и использовать как обычный класс

In [6]:
print(Foo)
print(Foo.bar)
f = Foo()
print(f)
print(f.bar)

<class '__main__.Foo'>
True
True


Конечно, можно от него наследовать:
```python
class FooChild(Foo):
    pass
```
превратится в:

In [ ]:
FooChild = type('FooChild', (Foo,), {})
print(FooChild)
print(FooChild.bar)  # bar is inherited from Foo

В какой-то момент вам захочется добавить методов вашему классу.
<br>Для этого просто определите функцию с нужной сигнатурой и присвойте её в качестве атрибута:

In [ ]:
def echo_bar(self):
    print(self.bar)

Foo = type('Foo', (), {'bar':True})
FooChild = type('FooChild', (Foo,), {'echo_bar': echo_bar})
hasattr(Foo, 'echo_bar')
hasattr(FooChild, 'echo_bar')

my_foo = FooChild()
my_foo.echo_bar()

Уже понятно, к чему я клоню: в Python классы являются объектами и можно создавать классы на ходу.

Это именно то, что Python делает, когда используется ключевое слово `class`, и делает он это с помощью метаклассов.

### Что такое метакласс (наконец)

Метакласс это "штука", которая создаёт классы.

Мы создаём класс для того, чтобы создавать объекты, так? А классы являются объектами. Метакласс это то, что создаёт эти самые объекты. Они являются классами классов, можно представить это себе следующим образом:
```python
MyClass = MetaClass()
MyObject = MyClass()
```
Мы уже видели, что `type` позволяет делать что-то в таком духе:
```python
MyClass = type('MyClass', (), {})
```

Это потому что функция `type` на самом деле является метаклассом. `type` это метакласс, который Питон внутренне использует для создания всех классов.

Естественный вопрос: с чего это он его имя пишется в нижнем регистре, а не `Type`?

Я полагаю, это просто для соответствия `str`, классу для создания объектов-строк, и `int`, классу для создания объектов-целых чисел. `type` это просто класс для создания объектов-классов.

Это легко проверить с помощью атрибута `__class__`:

В питоне всё (вообще всё!) является объектами. В том числе числа, строки, функции и классы — они все являются объектами и все были созданы из класса:

In [7]:
age = 35
age.__class__

name = 'bob'
name.__class__

def foo():
    pass
foo.__class__

class Bar: 
    pass
b = Bar()
b.__class__

int

str

function

__main__.Bar

А какой же `__class__` у каждого `__class__`?

In [ ]:
age.__class__.__class__
name.__class__.__class__
foo.__class__.__class__
b.__class__.__class__

Итак, метакласс это просто штука, создающая объекты-классы.

Если хотите, можно называть его "фабрикой классов"

`type` это встроенный метакласс, который использует Питон, но вы, конечно, можете создать свой.

### Атрибут `__metaclass__`

При написании класса можно добавить атрибут `__metaclass__`:
```python
class Foo:
  __metaclass__ = something
  ...
```
В таком случае Питон будет использовать указанный метакласс при создании класса `Foo`.
<br>Осторожно, тут есть тонкость!

Хоть вы и пишете `class Foo`, объект-класс пока ещё не создаётся в памяти.

Питон будет искать `__metaclass__` в определении класса. Если он его найдёт, то использует для создания класса `Foo`. Если же нет, то будет использовать `type`.

То есть когда вы пишете
```python
class Foo(Bar):
  pass
```

Питон делает следующее:
* Есть ли у класса `Foo` атрибут `__metaclass__`?
* Если да, создаёт в памяти объект-класс с именем `Foo`, используя то, что указано в `__metaclass__`.
* Если Питон не находит `__metaclass__`, он ищет `__metaclass__` в родительском классе `Bar` и попробует сделать то же самое.
* Если же `__metaclass__` не находится ни в одном из родителей, Питон будет искать `__metaclass__` на уровне модуля.
* И если он не может найти вообще ни одного `__metaclass__`, он использует `type` для создания объекта-класса.

Теперь важный вопрос: что можно положить в `__metaclass__`?

Ответ: что-нибудь, что может создавать классы.

А что создаёт классы? `type` или любой его подкласс, а также всё, что использует их.

### Пользовательские метаклассы

Основная цель метаклассов — автоматически изменять класс в момент создания.

Обычно это делает для API, когда хочется создавать классы в соответсвии с текущим контекстом.

Представим глупый пример: вы решили, что у всех классов в вашем модуле имена атрибутов должны быть записать в верхнем регистре. Есть несколько способов это сделать, но один из них — задать `__metaclass__` на уровне модуля.

В таком случае все классы этого модуля будут создаваться с использованием указанного метакласса, а нам остаётся только заставить метакласс переводить имена всех атрибутов в верхний регистр.

К счастью, `__metaclass__` может быть любым вызываемым объектом, не обязательно формальным классом
<br>_(я знаю, что-то со словом "класс" в названии не обязано быть классом, что за ерунда? Однако это полезно)._

Так что мы начнём с простого примера, используя функцию.

In [8]:
# метаклассу автоматически придёт на вход те же аргументы, которые обычно используются в `type`
def upper_attr(future_class_name: str, future_class_parents: tuple, future_class_attr: dict):
    """
    Возвращает объект-класс, имена атрибутов которого переведены в верхний регистр
    """
    # берём любой атрибут, не начинающийся с '__'
    attrs = [(name, value) for name, value in future_class_attr.items() if not name.startswith('__')]
    # переводим их в верхний регистр
    uppercase_attr = {name.upper(): value for name, value in attrs}

    # создаём класс с помощью `type`
    return type(future_class_name, future_class_parents, uppercase_attr)

class Foo(metaclass=upper_attr): 
    bar = 'bip'

hasattr(Foo, 'bar')
hasattr(Foo, 'BAR')

f = Foo()
f.BAR

False

True

'bip'

А теперь то же самое, только используя настояший класс:

In [ ]:
# помним, что `type` это на само деле класс, как `str` и `int`,
# так что от него можно наследовать
class UpperAttrMetaClass(type): 
    # Метод __new__ вызывается перед __init__
    # Этот метод создаёт объект и возвращает его,
    # в то время как __init__ просто инициализирует объект, переданный в качестве аргумента.
    # Обычно вы не используете __new__, если только не хотите проконтролировать, как объект создаётся
    # В данном случае созданный объект это класс, и мы хотим его настроить, поэтому мы перегружаем __new__.
    # Можно также сделать что-нибудь в __init__, если хочется.
    # В некоторых более продвинутых случаях также перегружается __call__, но этого мы сейчас не увидим.
    def __new__(
        upperattr_metaclass: UpperAttrMetaClass,
        future_class_name: str, 
        future_class_parents: tuple, 
        future_class_attr: dict,
    ):
        attrs = [(name, value) for name, value in future_class_attr.items() if not name.startswith('__')]
        uppercase_attr = {name.upper(): value for name, value in attrs}
        return type(future_class_name, future_class_parents, uppercase_attr)

class Foo(metaclass=UpperAttrMetaClass): 
    bar = 'bip'

f = Foo()
f.BAR    

Но это не совсем ООП. Мы напрямую вызываем `type` и не перегружаем вызов `__new__` родителя. Давайте сделаем это:

In [ ]:
class UpperAttrMetaclass(type): 
    def __new__(
        upperattr_metaclass: UpperAttrMetaClass, 
        future_class_name: str, 
        future_class_parents: tuple, 
        future_class_attr: dict,
    ):
        attrs = [(name, value) for name, value in future_class_attr.items() if not name.startswith('__')]
        uppercase_attr = {name.upper(): value for name, value in attrs}
        # используем метод type.__new__
        # базовое ООП, никакой магии
        return type.__new__(upperattr_metaclass, future_class_name, future_class_parents, uppercase_attr)

class Foo(metaclass=UpperAttrMetaClass): 
    bar = 'bip'

f = Foo()
f.BAR    

Вы, возможно, заметили дополнительный аргумент `upperattr_metaclass`. Ничего особого в нём нет: метод всегда получает первым аргументом текущий экземпляр. Точно так же, как вы используете `self` в обычным методах.

Конечно, имена, которые я тут использовал, такие длинные для ясности, но как и `self`, есть соглашение об именовании всех этих аргументов. Так что реальный метакласс выгляит как-нибудь так:

In [ ]:
class UpperAttrMetaclass(type): 
    def __new__(cls, name, bases, dct):
        attrs = [(name, value) for name, value in dct.items() if not name.startswith('__')]
        uppercase_attr = {name.upper(): value for name, value in attrs}
        return type.__new__(cls, name, bases, uppercase_attr)

class Foo(metaclass=UpperAttrMetaClass): 
    bar = 'bip'

f = Foo()
f.BAR   

Можно сделать даже лучше, использовав `super`, который вызовет наследование (поскольку, конечно, можно создать метакласс, унаследованный от метакласса, унаследованного от `type`):

In [ ]:
class UpperAttrMetaclass(type): 
    def __new__(cls, name, bases, dct):
        attrs = [(name, value) for name, value in dct.items() if not name.startswith('__')]
        uppercase_attr = {name.upper(): value for name, value in attrs}
        return super().__new__(cls, name, bases, uppercase_attr)

class Foo(metaclass=UpperAttrMetaClass): 
    bar = 'bip'

f = Foo()
f.BAR    

Вот и всё. О метаклассах больше ничего и не сказать.

Причина сложности кода, использующего метаклассы, не в самих метаклассах. Она в том, что обычно метаклассы используются для всяких изощрённых вещей, основанных на интроспекции, манипуляцией наследованием, переменными вроде `__dict__` и тому подобном.

Действительно, метаклассы особенно полезны для всякой "чёрной магии", а, следовательно, сложных штук.

Но сами по себе они просты:
* __перехватить создание класса__
* __изменить класс__
* __вернуть модифицированный__

### Зачем использовать метаклассы вместо функций?

Поскольку `__metaclass__` принимает любой вызываемый объект, с чего бы вдруг использовать класс, если это очевидно сложнее?

Тому есть несколько причин:
* Назначение яснее. Когда вы видите `UpperAttrMetaclass(type)`, вы сразу знаете, что дальше будет.
* Можно использовать ООП. Метаклассы могут наследоваться от метаклассов, перегружая родитальские методы.
* Лучше структурированный код. Вы не будете использовать метаклассы для таких простых вещей, как в примере выше. Обычно это что-то сложное. Возможность создать несколько методов и сгруппировать их в одном классе очень полезна, чтобы сделать код более удобным для чтения.
* Можно использовать `__new__`, `__init__` и `__call__`. Конечно, обычно можно всё сделать в `__new__`, но некоторым комфортнее использовать `__init__`
* Они называются метаклассами, чёрт возьми! Это должно что-то значить!

### Зачем вообще использовать метаклассы?

Наконец, главный вопрос. С чего кому-то использовать какую-то непонятную (и способствующую ошибкам) фичу?

Ну, обычно и не надо использовать:
> Метаклассы это глубокая магия, о которой 99% пользователей даже не нужно задумываться. Если вы думаете, нужно ли вам их использовать — вам не нужно (люди, которым они реально нужны, точно знают, зачем они им, и не нуждаются в объяснениях, почему).
> <br> _~ Гуру Питона Тим Питерс_

Основное применение метаклассов это создание API. Типичный пример — Django ORM.

Она позволяет написать что-то в таком духе:
```python
class Person(models.Model):
    name = models.CharField(max_length=30)
    age = models.IntegerField()
```
Однако если вы выполните следующий код:
```python
guy = Person(name='bob', age='35')
print guy.age
```
вы получите не `IntegerField`, а `int`, причём значение может быть получено прямо из базы данных.

Это возможно, потому что `models.Model` определяет `__metaclass__`, который сотворит некую магию и превратит класс `Person`, который мы только что определили простым выражением в сложную привязку к базе данных.

Django делает что-то сложное выглядящим простым, выставляя наружу простое API и используя метаклассы, воссоздающие код из API и незаметно делающие всю работу.

### Напоследок

Во-первых, вы узнали, что классы это объекты, которые могут создавать экземпляры.

На самом деле, классы это тоже экземпляры. Экземпляры метаклассов.
```python
class Foo: 
    pass

id(Foo)  # 142630324
```

Всё что угодно является объектом в Питоне: экземпляром класса или экземпляром метакласса.

Кроме `type`.

`type` является собственным метаклассом. Это нельзя воспроизвести на чистом Питоне и делается небольшим читерством на уровне реализации.

Во-вторых, метаклассы сложны. Вам не нужно использовать их для простого изменения классов. Это можно делать двумя разными способами:
* руками
* декораторы классов

В 99% случаев, когда вам нужно изменить класс, лучше использовать эти два.

Но в 99% случаев вам вообще не нужно изменять классы 😉

***
## <font color='blue'><u>6. Домашнее задание</u></font><a id="6"></a>

Реализовать абстрактный класс валюты, с наследниками `Euro`, `Dollar`, `Rubble`.
<br>Курс пусть будет `1 EUR == 2 USD == 100 RUB`

Имплементировать методы из примеров ниже:
```python
print(
    f"Euro.course(Rubble)   ==> {Euro.course(Rubble)}\n"
    f"Dollar.course(Rubble) ==> {Dollar.course(Rubble)}\n"
    f"Rubble.course(Euro)   ==> {Rubble.course(Euro)}\n"
)
# Euro.course(Rubble)   ==> 100.0 RUB for 1 EUR
# Dollar.course(Rubble) ==> 50.0 RUB for 1 USD
# Rubble.course(Euro)   ==> 0.01 EUR for 1 RUB

e = Euro(100)
r = Rubble(100)
d = Dollar(200)

print(
    f"e = {e}\n"
    f"e.to(Dollar) = {e.to(Dollar)}\n"
    f"e.to(Rubble) = {e.to(Rubble)}\n"
    f"e.to(Euro)   = {e.to(Euro)}\n"
)
# e = 100 EUR
# e.to(Dollar) = 200.0 USD
# e.to(Rubble) = 10000.0 RUB
# e.to(Euro)   = 100.0 EUR

print(
    f"r = {r}\n"
    f"r.to(Dollar) = {r.to(Dollar)}\n"
    f"r.to(Euro)   = {r.to(Euro)}\n"
    f"r.to(Rubble) = {r.to(Rubble)}\n"
)
# r = 100 RUB
# r.to(Dollar) = 2.0 USD
# r.to(Euro)   = 1.0 EUR
# r.to(Rubble) = 100.0 RUB

print(
    f"e > r   ==> {e > r}\n"
    f"e == d  ==> {e == d}\n"
)
# e > r   ==> True
# e == d  ==> True

print(
    f"e + r  =>  {e + r}\n"
    f"r + d  =>  {r + d}\n"
    f"d + e  =>  {d + e}\n"
)
# e + r  =>  101.0 EUR
# r + d  =>  10100.0 RUB
# d + e  =>  400.0 USD

print(sum([Euro(i) for i in range(5)]))
# 10.0 EUR
```

<center>🐍</center>